### WebCrawling

- 웹페이지의 종류
    - 정적페이지 : 페이지의 데이터가 변경될때 URL이 변경 O
    - 동적페이지 : 페이지의 데이터가 변경될때 URL이 변경 x
-requests package
    - 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력:URL->DATA
    - requests 패키지 : URL ->DATA
    
### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용 
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용

### Naver Stock Data
   - Kaspi 지수
   - Kosdaq 지수
   - USD : 원달러 환율

In [2]:
import requests
import pandas as pd

In [3]:
# 1. 웹서비스를 분석 : 크롬 개발자도구 :URL

In [4]:
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=3"

In [ ]:
#2. request > response :JSON(str)

In [5]:
response = requests.get(url)
response

<Response [200]>

In [7]:
response.text[:200]

'[{"localTradedAt":"2022-07-06","closePrice":"2,292.01","compareToPreviousClosePrice":"-49.77","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-2.13","openPrice"'

In [ ]:
#3. JSON(str) >list,dict > DataFrame

In [10]:
data = response.json()
type(data), data[:1]

(list,
 [{'localTradedAt': '2022-07-06',
   'closePrice': '2,292.01',
   'compareToPreviousClosePrice': '-49.77',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-2.13',
   'openPrice': '2,330.11',
   'highPrice': '2,332.14',
   'lowPrice': '2,290.33'}])

In [14]:
df=pd.DataFrame(data)[["localTradedAt","closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-24,"2,366.60"
9,2022-06-23,"2,314.32"


In [ ]:
# 4.함수 만들기
#params : pagesize, page

In [17]:
def stock_price(pagesize, page):
    url = f"https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[["localTradedAt","closePrice"]]

In [18]:
df=stock_price(30 ,2)
df.tail(2)

,localTradedAt,closePrice
28,2022-05-11,"2,592.27"
29,2022-05-10,"2,596.56"


In [ ]:
#KOSDAQ 데이터 수집 코드 작성

In [ ]:
#1.웹서비스 분석 :URL

In [ ]:
url = "https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2"

In [ ]:
#2. request > response :JSON(str)

In [19]:
response = requests.get(url)
response

<Response [200]>

In [ ]:
#3. JSON(str) >list,dict > DataFrame

In [20]:
data = response.json()
type(data), data[:1]

(list,
 [{'localTradedAt': '2022-07-06',
   'closePrice': '2,292.01',
   'compareToPreviousClosePrice': '-49.77',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-2.13',
   'openPrice': '2,330.11',
   'highPrice': '2,332.14',
   'lowPrice': '2,290.33'}])

In [21]:
df=pd.DataFrame(data)[["localTradedAt","closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-24,"2,366.60"
9,2022-06-23,"2,314.32"


In [29]:
def stock_price(pagesize, page,code="KOSDAQ"):
    """This function is crwaling stock price from naver stock web page
    
    params
    ------
    code : str : KOSPI or KOSDAQ
    page : int : page number
    page_size : int : one page size
        
    return
    ------
    type : DataFame of pandas
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[["localTradedAt","closePrice"]]

In [26]:
stock_price(10,1,"KOSDAQ")

,localTradedAt,closePrice
0,2022-08-03,812.47
1,2022-08-02,804.34
2,2022-08-01,807.61
3,2022-07-29,803.62
4,2022-07-28,798.32
5,2022-07-27,795.70
6,2022-07-26,789.93
7,2022-07-25,789.69
8,2022-07-22,789.75
9,2022-07-21,795.15


In [27]:
kospi = stock_price(60,1,"KOSPI")
kosdaq = stock_price(60,1,"KOSDAQ")

In [30]:
# docstring : 함수를 사용하는 방법을 문자열로 작성
#help(), shift+tab(설명)
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSDAQ')
    This function is crwaling stock price from naver stock web page
    
    params
    ------
    code : str : KOSPI or KOSDAQ
    page : int : page number
    page_size : int : one page size
        
    return
    ------
    type : DataFame of pandas



### 원달러 환율 데이터 수집
- 주식 > 시장지표 > 미국USD>일별시세(더보기)

In [38]:
def exchage_rate(code="FX_USDKRW", page=1, page_size=60):
    url = f'https://api.stock.naver.com/marketindex/exchange/{code}/prices?page={page}\&pageSize={page_size}'
    response = requests.get(url)
    columns = ["localTradedAt", "closePrice"]
    datas = response.json()
    return pd.DataFrame(datas)[columns]

In [40]:
usd = exchage_rate()
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


In [ ]:
#데이터 분석
#상관관계 분석 : 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석방법
#원달러 환율이 높으면 코스피,코스닥 주가 지수가 낮다. >음의 상관관계를 갖는다.

In [ ]:
#피어슨 상관계수 : df.corr()
#1과 가까울수록 강한 양의 상관관계를 갖는다.
#-1과 가까울수록 강한 음의 상관관계를 갖는다.
#0과 가까울수록 관계가 없다

In [97]:
#데이터 전처리
df = kospi.copy()
df["kosdaq"] = kosdaq["closePrice"]
df["usd"] = usd["closePrice"]
df=df.rename(columns = {"closePrice" : "kospi"})
df.tail(2)

,localTradedAt,kospi,kosdaq,usd
58,2022-05-11,"2,592.27",866.34,"1,274.50"
59,2022-05-10,"2,596.56",856.14,"1,276.00"


In [98]:
df.dtypes

localTradedAt    object
kospi            object
kosdaq           object
usd              object
dtype: object

In [99]:
#컬럼의 데이터 타입 변경 : str > float
#df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력
df["kospi"]=df["kospi"].apply(lambda data: float(data.replace(",","")))
df["kosdaq"]=df["kosdaq"].apply(lambda data: float(data.replace(",","")))
df["usd"]=df["usd"].apply(lambda data: float(data.replace(",","")))

In [100]:
df.dtypes

localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object

In [101]:
df[['kospi','kosdaq','usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984042,-0.878345
kosdaq,0.984042,1.000000,-0.821311
usd,-0.878345,-0.821311,1.000000


In [ ]:
#copy(),apply,lambda

In [ ]:
#copy()

In [57]:
data1 = [1,2,3]
data2=data1          #얕은 복사 : 주소값 복사
data3=data1.copy()   #깊은복사 : 값 복사
print(data1,data2,data3)
data1[1] =4
print(data1,data2,data3)

[1, 2, 3] [1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


In [ ]:
#apply(func) : 모든 데이터를 func을 적용시킨 결과 출력

In [60]:
df = pd.DataFrame([{"age":23}, {"age":36}, {"age":27}])
df

,age
0,23
1,36
2,27


In [83]:
#연령대 컬럼을 추가
def change_ages(age):
    return age//10 * 10

In [84]:
df["age"]

0    23
1    36
2    27
Name: age, dtype: int64

In [85]:
df["ages"]=df["age"].apply(change_ages)
df

,age,ages
0,23,20
1,36,30
2,27,20


In [ ]:
#lambda :일회성 함수
#사용이유 : 간단한 함수를 메모리를 절약하여 사용

In [90]:
def plus(n1,n2):
    return n1 + n2

def minus(n1,n2):
    return n1 - n2

def calc(func,n1,n2):
    return func(n1, n2)

calc(plus,1,2), calc(minus,1,2)

(3, -1)

In [89]:
plus_lambda = lambda n1,n2 : n1 +n2
plus(2,3), plus_lambda(2,3)

(5, 5)

In [ ]:
def calc(func,n1,n2):
    return func(n1, n2)


In [92]:
calc(lambda n1,n2 : n1 +n2,1,2), calc(lambda n1,n2 : n1 -n2,1,2)

(3, -1)